In [ ]:
import sys, argparse, logging
import pymssql
import _mssql
import datetime
from decimal import Decimal
import pymongo
from pymongo import MongoClient
import re

from tmqr.settings import *

try:
    from tmqr.settings_local import *
except:
    pass


In [ ]:
# Init mongo asset index
client = MongoClient(MONGO_CONNSTR)
mongo_db = client[MONGO_DB]
mongo_collection = mongo_db['asset_index']

In [ ]:
mongo_collection.create_index([('contr', pymongo.ASCENDING), 
                               ('mkt', pymongo.ASCENDING), 
                               ('type', pymongo.ASCENDING)])

In [ ]:
mongo_collection.create_index([('instr', pymongo.ASCENDING), 
                               ('type', pymongo.ASCENDING),
                               ('exp', pymongo.ASCENDING)])

In [ ]:
%%timeit
mongo_collection.find_one({'tckr': 'US.F.CLM83.830520'})

In [ ]:
%%timeit
mongo_collection.find_one({'tckr': 'US.P.F-GCM18-180627.180425@860.0'})

In [ ]:
len('US.P.F-GCM18-180627.180425@860.0')

In [ ]:
%%timeit
regx = re.compile("^US.F.CLM83.*", re.IGNORECASE)
mongo_collection.find_one({'contr':  "GCM18", 'mkt': 'US', 'type': 'F'})
#mongo_collection.find_one({'tickr':  regx})

In [ ]:
%%timeit
mongo_collection.find({'instr': 'US.CL', 'mkt': 'US', 'type': 'P', 'exp': {'$gt': datetime.datetime(2015, 5, 20, 0, 0)}}).count()

In [ ]:
mongo_collection.explain().find({'instr': 'US.CL', 'mkt': 'US', 'type': 'P', 'exp': {'$gt': datetime.datetime(2015, 5, 20, 0, 0)}}).count()

# Getting futures chains

In [ ]:
%timeit list(mongo_collection.find({'type':'F', 'instr': 'US.CL'}).sort('exp', 1))

In [ ]:
[x['tckr'] for x in mongo_collection.find({'type':'F', 
                                           'instr': 'US.CL', 
                                           'exp': {'$gt': datetime.datetime(2011, 1, 1)}}, 
                                          projection=['tckr']).sort('exp', 1)]